# Footprint the extents of rasters

In [1]:
import rasterio as rio
import rasterio.features
import rasterio.warp
from rasterio.plot import show
from rasterio import Affine, MemoryFile
from rasterio.enums import Resampling

import pandas as pd
import geopandas as gpd
import shapely
from shapely.geometry import box
import fiona
from fiona.crs import from_epsg
import pprint

import glob
import os
import s3fs

import sys
sys.path.append('/efs/pmontesa/code/geoscitools')
sys.path.append('/home/pmontesa/code/geoscitools')
import maplib
import footprintlib
from footprintlib import *

# https://gis.stackexchange.com/questions/375577/how-do-i-write-out-a-mixed-geometry-geodataframe-to-a-geopackage
# https://gis.stackexchange.com/questions/187877/how-to-polygonize-raster-to-shapely-polygons

### Set up dictionaries of parameters

Set up an AWSSession

In [3]:
if False:
    import configparser
    config = configparser.ConfigParser()
    config.read('/efs/pmontesa/credentials')
    print(config.sections()[0])
    [print(key) for key in config['boreal_pub']]

    #parse_aws_creds(credentials_fn)[0]

## Build a dictionary of the raster datasets for footprinting

In [18]:
dict_list = [

    {'location':'s3',  #'local' or 's3'
     'data_dir': "s3://terrapulse-pub-data/boreal-forest-data",
     's3_url_prefix': None,
     'data_source': 'terrapulse-pub-data',
     'image_type': 'forest_loss',#'tcc_pvalue',#'tcc_slope', #'tcc_2020', tcc_pvalue, forest_age, forest_loss
     'search_tail_string': '/**/*.tif',
     'search_link_string': '',
     'DO_DATAMASK': False,
     'GET_ONLY_DATASETMASK': True,
     'out_footprint_dir': '/efs/projects/forest_height',
     'footprint_name_SPLIT_CHAR': '_',
     'footprint_name_SPLIT_POS': 0,
     'aws_credential_fn': '/efs/pmontesa/credentials'
    },
    {'location':'local', #'local' or 's3'
     'data_dir': "/efs/projects/data/hls_tiles",
     's3_url_prefix': None,
     'data_source': 'hls_tiles',
     'image_type': 'boreal_tiles_v0003',
     'search_tail_string': '/*.tif',
     'search_link_string': '',
     'DO_DATAMASK': False,
     'GET_ONLY_DATASETMASK': True,
     'out_footprint_dir': '/efs/projects/forest_height/test_out',
     'footprint_name_SPLIT_CHAR': '_',
     'footprint_name_SPLIT_POS': 0,
     'aws_credential_fn': None
    },
    {'location':'local', #'local' or 's3'
     'data_dir': "/att/nobackup/pmontesa/userfs02/data/worldclim/1km/bioclim",
     's3_url_prefix': None,
     'data_source': 'worldclim',
     'image_type': 'wc2.0_30s_bio',
     'search_tail_string': '/wc2.0_bio_30s*.tif',
     'search_link_string': '',
     'DO_DATAMASK': False,
     'GET_ONLY_DATASETMASK': True,
     'out_footprint_dir': '/att/nobackup/pmontesa/userfs02/projects/ilab/boreal_height',
     'footprint_name_SPLIT_CHAR': '_',
     'footprint_name_SPLIT_POS': 0,
     'aws_credential_fn': None
    },   
    {'location':'local', #'local' or 's3'
     'data_dir': "/att/nobackup/pmontesa/userfs02/data",
     's3_url_prefix': None,
     'data_source': 'soil_grids',
     'image_type': 'soil_grids',
     'search_tail_string': '/*.tif',
     'search_link_string': '',
     'DO_DATAMASK': False,
     'GET_ONLY_DATASETMASK': True,
     'out_footprint_dir': '/att/nobackup/pmontesa/userfs02/projects/ilab/boreal_height',
     'footprint_name_SPLIT_CHAR': '_',
     'footprint_name_SPLIT_POS': 0,
     'aws_credential_fn': None
    }, 
    {'location':'local', #'local' or 's3'
     'data_dir': "/att/nobackup/pmontesa/userfs02/data",
     's3_url_prefix': None,
     'data_source': 'awi',
     'image_type': 'MAGT',
     'search_tail_string': '/*PERPROB*.tif',
     'search_link_string': '',
     'DO_DATAMASK': False,
     'GET_ONLY_DATASETMASK': True,
     'out_footprint_dir': '/att/nobackup/pmontesa/userfs02/projects/ilab/boreal_height',
     'footprint_name_SPLIT_CHAR': '_',
     'footprint_name_SPLIT_POS': 0,
     'aws_credential_fn': None
    },    
    {'location': 'local',
     'data_dir': "/adapt/nobackup/projects/ilab/scratch/jli30/data/boreal_forest",
     's3_url_prefix': None,
     'data_source': 'modis_trend',
     'image_type': 'modis_trend',
     'search_tail_string': '/*peak.bias.tif',
     'search_link_string': '',
     'DO_DATAMASK': False,
     'GET_ONLY_DATASETMASK': True,
     'out_footprint_dir': '/att/nobackup/pmontesa/userfs02/projects/ilab/boreal_height',
     'footprint_name_SPLIT_CHAR': '_',
     'footprint_name_SPLIT_POS': 0,
     'aws_credential_fn': None
    },
    {'location': 's3',
     'data_dir': "s3://esa-worldcover",
     's3_url_prefix': "https://esa-worldcover.s3.eu-central-1.amazonaws.com",
     'data_source': 'v100/2020',
     'image_type': 'v100/2020/map',
     'search_tail_string': '/*Map.tif',
     'search_link_string': '',
     'DO_DATAMASK': False,
     'GET_ONLY_DATASETMASK': True,
     'out_footprint_dir': '/efs/projects/forest_height',
     'footprint_name_SPLIT_CHAR': '_',
     'footprint_name_SPLIT_POS': 0,
     'aws_credential_fn': None
    }
]

In [5]:
if False:
    for d in dict_list:
        print('\n', d['data_source'])

        if d['location'] == 's3':
            import s3fs
            # Use s3fs to access file remote s3 system
            s3 = footprintlib.get_s3_fs_from_creds('/home/pmontesa/credentials_test') # '/efs/pmontesa/credentials'
            print(s3.ls(d['data_dir']))

### Run the footprinting code over an s3 datalist to return and geodataframe and a geopackage **updated** 

https://stackoverflow.com/questions/67812512/rasterio-does-not-exist-in-the-file-system-and-is-not-recognized-as-a-support

In [19]:
######################
# Select a dictionary
d = dict_list[0]
TEST=False
######################

In [ ]:
%%time

footprint_gdf_list = list() 
filename = 'footprints_' + d['data_source'].replace('/','_')+'_'+d['image_type'].replace('/','_')+'.gpkg'

#SEARCH_STR_LIST = ["/", sensor, image_type, ".tif"]
SEARCH_STR_LIST = ["/", d['image_type'], d['search_tail_string']]
keystring = d['search_link_string'].join(SEARCH_STR_LIST)
print(d['data_dir'] + keystring)

# Build raster filename list
if d['location'] == 'local':
    r_fn_list = glob.glob(d['data_dir'] + keystring)
    
if d['location'] == 's3':
    print('on s3...')
    if d['aws_credential_fn'] is None:
        s3 = get_s3_fs_from_creds(anon=True)
    else:
        s3 = get_s3_fs_from_creds(credentials_fn=d['aws_credential_fn'])
    r_fn_list = s3.glob(d['data_dir'] + keystring)
    r_fn_list = ['s3://' + f for f in r_fn_list]

if TEST:
    r_fn_list = r_fn_list[0:2]
    print(r_fn_list)
    filename = 'TEST_' + filename


outprint_fn = os.path.join(d['out_footprint_dir'], filename)

print(f'{len(r_fn_list)} total inputs from ' + d['data_source'] )    

# Get list of geodataframes of individual raster footprints 
#
if d['location'] == 'local':
    # Raster files are local
    # Get a list geodataframes of indiv raster footprint geometries (both data and mask)
    f_gdf_lst = [raster_footprint(r_fn, DO_DATAMASK=d['DO_DATAMASK'], GET_ONLY_DATASETMASK=d['GET_ONLY_DATASETMASK'], R_READ_MODE='r') for r_fn in r_fn_list]
    
if d['location'] == 's3':
    # Raster files are on s3
    # Get the rio AWS session needed to access the s3 rasters in order to footprint
    if d['aws_credential_fn'] is None:
        if d['s3_url_prefix'] is not None:
            # This doesnt work
            #f_gdf_lst = [raster_footprint(s3.open(r_fn, mode = 'r'), DO_DATAMASK=d['DO_DATAMASK'], GET_ONLY_DATASETMASK=d['GET_ONLY_DATASETMASK'], R_READ_MODE='r') for r_fn in r_fn_list]
            r_fn_list = [f.replace(d['data_dir'], d['s3_url_prefix']) for f in r_fn_list]
        f_gdf_lst = [raster_footprint(r_fn, DO_DATAMASK=d['DO_DATAMASK'], GET_ONLY_DATASETMASK=d['GET_ONLY_DATASETMASK'], R_READ_MODE='r') for r_fn in r_fn_list]
    else:
        aws_session = get_rio_aws_session_from_creds(d['aws_credential_fn'])
        with rio.Env(aws_session):
            # Get a list geodataframes of indiv raster footprint geometries (both data and mask)
            f_gdf_lst = [raster_footprint(r_fn, DO_DATAMASK=d['DO_DATAMASK'], GET_ONLY_DATASETMASK=d['GET_ONLY_DATASETMASK'], R_READ_MODE='r') for r_fn in r_fn_list]
        
if len(f_gdf_lst)==0:
    print('\nNothing was footprinted.\n')
else:
    # Build a footprint database geodataframe from all individual raster footprints
    footprint_gdf = build_footprint_db(f_gdf_lst, TO_GCS=False, WRITE_GPKG=True, OUT_F_NAME=outprint_fn, OUT_LYR_NAME=d['image_type'])

    footprint_gdf['footprint_name'] = footprint_gdf['file'].str.split(d['footprint_name_SPLIT_CHAR'], expand=True)[d['footprint_name_SPLIT_POS']]
    footprint_gdf_list.append(footprint_gdf)


s3://terrapulse-pub-data/boreal-forest-data/forest_loss/**/*.tif
on s3...


In [8]:
r_fn_list

['s3://esa-worldcover/v100/2020/map/ESA_WorldCover_10m_2020_v100_N00E006_Map.tif',
 's3://esa-worldcover/v100/2020/map/ESA_WorldCover_10m_2020_v100_N00E009_Map.tif']

In [45]:
with rasterio.Env(AWS_NO_SIGN_REQUEST='YES'):
    with rasterio.open(r_fn_list[0], mode='r') as dataset:
        print(dataset.profile)

{'driver': 'GTiff', 'dtype': 'uint8', 'nodata': 0.0, 'width': 36000, 'height': 36000, 'count': 1, 'crs': CRS.from_epsg(4326), 'transform': Affine(8.333333333333333e-05, 0.0, 6.0,
       0.0, -8.333333333333333e-05, 3.0), 'blockxsize': 1024, 'blockysize': 1024, 'tiled': True, 'compress': 'deflate', 'interleave': 'band'}


#### Test access to rasters on s3

In [ ]:
test_terraPulse_file = r_fn_list[1]

import rioxarray as rxr

with rio.Env(aws_session):
    #arr= rxr.open_rasterio(test_terraPulse_file,  masked=False)
    
    with rio.open(test_terraPulse_file) as src:
        profile = src.profile
        arr = src.read(1)
        #arr.plot()
        from rasterio.plot import show_hist
        show_hist(
            src, bins=50, lw=0.0, stacked=False, alpha=0.3,
            histtype='stepfilled', title="Histogram")

### Select the footprints gdf

In [41]:
footprint_gdf = footprint_gdf_list[0]
#print(footprint_gdf.path.to_list())
#print(footprint_gdf.to_crs(4326).iloc[1])
#print(footprint_gdf.iloc[1])
#footprint_gdf[footprint_gdf.file=='MCD12Q2.h12v02.peak.bias.tif'].to_crs(4326).geometry.to_json()
#footprint_gdf.crs.to_json()

## Map the footprints

In [42]:
#maplib.MAP_LAYER_FOLIUM(footprint_gdf.iloc[[3,5]], zoom_start=2)
maplib.MAP_LAYER_FOLIUM(footprint_gdf, LAYER_COL_NAME='file', zoom_start=2, lon_start=0)

### Check terrapulse stand age footprints

In [6]:
f_fn = '/efs/projects/forest_height/footprints_terrapulse-pub-data_forest_age_v2.gpkg'
import fiona
print(fiona.listlayers(f_fn))

f_age_gdf = gpd.read_file(f_fn, layer=fiona.listlayers(f_fn)[0])
maplib.MAP_LAYER_FOLIUM(f_age_gdf, LAYER_COL_NAME='path', zoom_start=2, lon_start=0)

['forest_age']


/srv/conda/envs/notebook/lib/python3.8/site-packages/geopandas/geodataframe.py:577: RuntimeWarning: Sequential read of iterator was interrupted. Resetting iterator. This can negatively impact the performance.
  for feature in features_lst:


### Manually check footprint for a single image

In [30]:
r_fn = '/att/nobackup/pmontesa/userfs02/data/MAGT/UiO_PEX_PERPROB_3.0_20171201_2000_2016.tif'
r_fn = '/att/nobackup/pmontesa/userfs02/data/soil_grids/CECSOL_M_sl2_250m_ll.tif'
r_fn = '/att/nobackup/pmontesa/userfs02/data/worldclim/1km/bioclim/wc2.0_30s_bio/wc2.0_bio_30s_01.tif'
footprint_gdf = raster_footprint(r_fn, DO_DATAMASK=d['DO_DATAMASK'], GET_ONLY_DATASETMASK=d['GET_ONLY_DATASETMASK'], R_READ_MODE='r+')
maplib.MAP_LAYER_FOLIUM(footprint_gdf, LAYER_COL_NAME='path', zoom_start=2, lon_start=0)